In [46]:
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.preprocessing import PolynomialFeatures
import warnings
warnings.filterwarnings('ignore')

from plotnine import *

In [3]:
read_file = pd.read_excel ("MSN Codes.xlsx")
read_file.to_csv ("MSN Codes.csv", index = None, header=True)
read_file = pd.read_excel ("Investment_Data_Train.xlsx")
read_file.to_csv ("Investment_Data_Train.csv", index = None, header=True)
# read_file = pd.read_excel ("Investment_Data_2020.xlsx")
# read_file.to_csv ("Investment_Data_2020.csv", index = None, header=True)

In [4]:
msn_code = pd.read_csv('MSN Codes.csv')

investment_data = pd.read_csv("Investment_Data_Train.csv")

In [6]:
single_variables = investment_data[[
    'StateCode', 'Year', 'State',
    'CO2 Emissions (Mmt)', 'TotalNumberofInvestments', 'TotalAmountofAssistance'
]].drop_duplicates()

multi_variables = investment_data[[
    'StateCode', 'Year', 'State',
    'MSN', 'Amount'
]].drop_duplicates()

MSN_feature_pivot = multi_variables.pivot(index=['StateCode', 'Year', 'State'], columns='MSN', values='Amount')
# MSN_feature_pivot[MSN_feature_pivot.isna().any(axis=1)]

tidy_data = MSN_feature_pivot.merge(single_variables, left_on=['StateCode', 'Year'], right_on=['StateCode', 'Year'])
tidy_data_wo_na_response = tidy_data[~tidy_data['TotalAmountofAssistance'].isna()].drop(columns=['State'])
tidy_data_wo_na_response

,StateCode,Year,BDFDB,BDPRP,BFFDB,BFPRP,CLPRB,CLPRK,CLPRP,COPRK,...,TETCB,WDEXB,WDPRB,WDTCB,WSTCB,WWPRB,WYTCB,CO2 Emissions (Mmt),TotalNumberofInvestments,TotalAmountofAssistance
0,AK,2015,21.0,4.0,21.0,4.0,17747.0,15.073,1177.0,5.717,...,614242.0,0.0,6865.0,6865.0,622.0,7488.0,1488.0,35.027804,16.0,3345612.0
1,AK,2016,27.0,5.0,27.0,5.0,13942.0,14.957,932.0,5.722,...,593662.0,0.0,7636.0,7636.0,412.0,8048.0,1563.0,33.405611,14.0,2247734.0
2,AK,2017,29.0,5.0,29.0,5.0,14365.0,14.978,959.0,5.723,...,601492.0,0.0,6501.0,6501.0,436.0,6937.0,1305.0,33.729130,18.0,13509662.0
3,AK,2018,15.0,3.0,15.0,3.0,13752.0,15.253,902.0,5.706,...,604644.0,0.0,6910.0,6910.0,456.0,7366.0,1411.0,34.514589,44.0,18236626.0
4,AK,2019,0.0,0.0,0.0,0.0,14867.0,15.252,975.0,5.698,...,605689.0,0.0,6418.0,6418.0,386.0,6803.0,1270.0,34.259440,32.0,13734021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,WY,2015,0.0,0.0,1192.0,209.0,6538240.0,17.399,375773.0,5.717,...,507121.0,0.0,4904.0,4904.0,2.0,4905.0,35009.0,64.092863,18.0,249960.0
256,WY,2016,0.0,0.0,0.0,0.0,5169939.0,17.394,297218.0,5.722,...,504984.0,0.0,4357.0,4357.0,2.0,4358.0,40522.0,61.259993,23.0,1359346.0
257,WY,2017,0.0,0.0,0.0,0.0,5516757.0,17.433,316454.0,5.723,...,534840.0,0.0,5049.0,5049.0,0.0,5049.0,39806.0,62.796487,47.0,413987.0
258,WY,2018,0.0,0.0,0.0,0.0,5315953.0,17.476,304188.0,5.706,...,557604.0,0.0,4934.0,4934.0,0.0,4934.0,36936.0,63.846081,53.0,573639.0


In [9]:
tidy_input_data = tidy_data_wo_na_response.drop(columns=['TotalAmountofAssistance'])
tidy_response_data = tidy_data_wo_na_response[['StateCode', 'Year', 'TotalAmountofAssistance']]

tidy_input_data

,StateCode,Year,BDFDB,BDPRP,BFFDB,BFPRP,CLPRB,CLPRK,CLPRP,COPRK,...,TEPRB,TETCB,WDEXB,WDPRB,WDTCB,WSTCB,WWPRB,WYTCB,CO2 Emissions (Mmt),TotalNumberofInvestments
0,AK,2015,21.0,4.0,21.0,4.0,17747.0,15.073,1177.0,5.717,...,1429942.0,614242.0,0.0,6865.0,6865.0,622.0,7488.0,1488.0,35.027804,16.0
1,AK,2016,27.0,5.0,27.0,5.0,13942.0,14.957,932.0,5.722,...,1435363.0,593662.0,0.0,7636.0,7636.0,412.0,8048.0,1563.0,33.405611,14.0
2,AK,2017,29.0,5.0,29.0,5.0,14365.0,14.978,959.0,5.723,...,1454196.0,601492.0,0.0,6501.0,6501.0,436.0,6937.0,1305.0,33.729130,18.0
3,AK,2018,15.0,3.0,15.0,3.0,13752.0,15.253,902.0,5.706,...,1410604.0,604644.0,0.0,6910.0,6910.0,456.0,7366.0,1411.0,34.514589,44.0
4,AK,2019,0.0,0.0,0.0,0.0,14867.0,15.252,975.0,5.698,...,1370342.0,605689.0,0.0,6418.0,6418.0,386.0,6803.0,1270.0,34.259440,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,WY,2015,0.0,0.0,1192.0,209.0,6538240.0,17.399,375773.0,5.717,...,9080777.0,507121.0,0.0,4904.0,4904.0,2.0,4905.0,35009.0,64.092863,18.0
256,WY,2016,0.0,0.0,0.0,0.0,5169939.0,17.394,297218.0,5.722,...,7518440.0,504984.0,0.0,4357.0,4357.0,2.0,4358.0,40522.0,61.259993,23.0
257,WY,2017,0.0,0.0,0.0,0.0,5516757.0,17.433,316454.0,5.723,...,7794586.0,534840.0,0.0,5049.0,5049.0,0.0,5049.0,39806.0,62.796487,47.0
258,WY,2018,0.0,0.0,0.0,0.0,5315953.0,17.476,304188.0,5.706,...,7711992.0,557604.0,0.0,4934.0,4934.0,0.0,4934.0,36936.0,63.846081,53.0


In [21]:
filtered_MSN_lst = ['BFPRP', 'CLPRB', 'ENPRP', 'GETCB', 'HYTCB', 'NGMPB', 'NUETB', 'PAPRB', 'REPRB', 'SOTCB', 'TETCB', 'WWPRB', 'WYTCB']

corr_tidy = tidy_data_wo_na_response.corr(method = 'spearman', numeric_only=True)
corr_last3 = corr_tidy.iloc[:, -3:]
corr_last3.loc[(corr_last3['TotalAmountofAssistance'] > 0.1) ]
MSN_filtered_df = corr_tidy.iloc[:, -3:].reset_index()
MSN_filtered_df[MSN_filtered_df['index'].isin(filtered_MSN_lst)]

,index,CO2 Emissions (Mmt),TotalNumberofInvestments,TotalAmountofAssistance
4,BFPRP,0.419783,0.389891,0.305276
5,CLPRB,0.317560,-0.009295,-0.152906
10,ENPRP,0.377371,0.345420,0.252728
11,GETCB,0.416013,0.255252,0.247244
12,HYTCB,0.094866,0.272681,0.218094
14,NGMPB,0.366838,0.110675,-0.049101
17,NUETB,0.664974,0.289534,0.323171
18,PAPRB,0.297733,0.098260,-0.064810
20,REPRB,0.536408,0.487344,0.419597
21,SOTCB,0.366796,0.055292,0.210624


In [29]:
filtered_columns = ['StateCode', 'Year'] + filtered_MSN_lst + ['CO2 Emissions (Mmt)', 'TotalNumberofInvestments']

filtered_input_data = tidy_input_data[filtered_columns]
stateCodes = filtered_input_data.StateCode.unique()
filtered_input_data

,StateCode,Year,BFPRP,CLPRB,ENPRP,GETCB,HYTCB,NGMPB,NUETB,PAPRB,REPRB,SOTCB,TETCB,WWPRB,WYTCB,CO2 Emissions (Mmt),TotalNumberofInvestments
0,AK,2015,4.0,17747.0,0.0,186.0,14621.0,380887.0,0.0,1007501.0,23807.0,3.0,614242.0,7488.0,1488.0,35.027804,16.0
1,AK,2016,5.0,13942.0,0.0,186.0,15317.0,369865.0,0.0,1026401.0,25155.0,14.0,593662.0,8048.0,1563.0,33.405611,14.0
2,AK,2017,5.0,14365.0,0.0,186.0,15143.0,382940.0,0.0,1033270.0,23620.0,20.0,601492.0,6937.0,1305.0,33.729130,18.0
3,AK,2018,3.0,13752.0,0.0,186.0,15151.0,375279.0,0.0,997415.0,24158.0,28.0,604644.0,7366.0,1411.0,34.514589,44.0
4,AK,2019,0.0,14867.0,0.0,186.0,14456.0,364363.0,0.0,968358.0,22754.0,40.0,605689.0,6803.0,1270.0,34.259440,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,WY,2015,209.0,6538240.0,209.0,663.0,8091.0,1998295.0,0.0,494355.0,49887.0,27.0,507121.0,4905.0,35009.0,64.092863,18.0
256,WY,2016,0.0,5169939.0,0.0,663.0,8987.0,1878141.0,0.0,415795.0,54565.0,33.0,504984.0,4358.0,40522.0,61.259993,23.0
257,WY,2017,0.0,5516757.0,0.0,663.0,10352.0,1788258.0,0.0,433655.0,55916.0,45.0,534840.0,5049.0,39806.0,62.796487,47.0
258,WY,2018,0.0,5315953.0,0.0,663.0,8883.0,1842525.0,0.0,502031.0,51483.0,67.0,557604.0,4934.0,36936.0,63.846081,53.0


In [62]:
X_variables = filtered_input_data#.drop(columns=['StateCode', 'Year'])
y_variables = tidy_response_data#.drop(columns=['StateCode', 'Year'])

train_X = X_variables[X_variables.Year < 2019]
train_y = y_variables[y_variables.Year < 2019]
test_X = X_variables[X_variables.Year == 2019]
compare_results = y_variables[y_variables.Year == 2019]

In [78]:
compare_results_total = None

for state in stateCodes:
    state_train_X = train_X[train_X['StateCode']==state].drop(columns=['StateCode', 'Year'])
    state_train_y = train_y[train_y['StateCode']==state].drop(columns=['StateCode', 'Year'])
    state_test_X = test_X[test_X['StateCode']==state].drop(columns=['StateCode', 'Year'])
    
#     poly = PolynomialFeatures(2, interaction_only=False)
#     poly.fit(state_train_X)
#     poly_train_X = poly.transform(state_train_X)
#     poly_test_X = poly.transform(state_test_X)

    poly_train_X = state_train_X
    poly_test_X = state_test_X
    
    scaler = StandardScaler()
    scaler.fit(poly_train_X)
    poly_train_X = scaler.transform(poly_train_X)
    poly_test_X = scaler.transform(poly_test_X)
    
    regr = ElasticNet(random_state=0, l1_ratio=0.3, alpha = 10)
    regr.fit(poly_train_X, state_train_y)
    predictions = regr.predict(poly_test_X)
    
    compare_results_poly = compare_results[compare_results['StateCode']==state]
    compare_results_poly['predict'] = predictions.tolist()
    compare_results_poly['ans_div_predict'] = compare_results_poly['TotalAmountofAssistance'] / compare_results_poly['predict']
    
    if compare_results_total is None:
        compare_results_total = compare_results_poly
    else:
        compare_results_total = pd.concat([compare_results_total, compare_results_poly])
    
print('RSME: ', ((compare_results_total.predict - compare_results_total.TotalAmountofAssistance) ** 2).mean() ** .5)
compare_results_total


RSME:  45043573.330839075


,StateCode,Year,TotalAmountofAssistance,predict,ans_div_predict
4,AK,2019,13734021.0,1.496306e+07,0.917862
9,AL,2019,840218.0,2.126279e+06,0.395159
14,AR,2019,60034678.0,4.975104e+07,1.206702
19,AZ,2019,95135122.0,2.396290e+06,39.700997
24,CA,2019,158244985.0,3.234098e+07,4.893017
29,CO,2019,10796826.0,2.961739e+07,0.364543
34,CT,2019,1306178.0,1.214841e+06,1.075184
44,DE,2019,1485254.0,1.200334e+06,1.237367
49,FL,2019,52936642.0,2.576857e+07,2.054310
54,GA,2019,2374660.0,1.065483e+07,0.222872


In [79]:
tidy_response_data[tidy_response_data['StateCode'] == 'AR']
investment_data[(investment_data['StateCode'] == 'AR')]

,Unnamed: 0,MSN,StateCode,Year,Amount,State,CO2 Emissions (Mmt),TotalNumberofInvestments,TotalAmountofAssistance
58,58,BDFDB,AR,2015,4429.0,Arkansas,59.053365,477.0,13453887.0
59,59,BDPRP,AR,2015,815.0,Arkansas,59.053365,477.0,13453887.0
60,60,BFFDB,AR,2015,4429.0,Arkansas,59.053365,477.0,13453887.0
61,61,BFPRP,AR,2015,815.0,Arkansas,59.053365,477.0,13453887.0
62,62,CLPRB,AR,2015,1819.0,Arkansas,59.053365,477.0,13453887.0
...,...,...,...,...,...,...,...,...,...
6158,6158,WDPRB,AR,2019,81001.0,Arkansas,65.072202,302.0,60034678.0
6159,6159,WDTCB,AR,2019,72168.0,Arkansas,65.072202,302.0,60034678.0
6160,6160,WSTCB,AR,2019,4040.0,Arkansas,65.072202,302.0,60034678.0
6161,6161,WWPRB,AR,2019,85041.0,Arkansas,65.072202,302.0,60034678.0
